In [1]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB

(cardZ, cardA, cardM, cardY) = (2,2,2,2)

Here needs $P_{ZAMY}$

In [2]:
P_ZAMY = {(0, 0, 0, 0): 0.336, (0, 0, 1, 0): 0.084, (0, 1, 0, 0): 0.056, (0, 1, 1, 0): 0.224, (1, 0, 0, 0): 0.096, (1, 0, 1, 0): 0.024, (1, 1, 0, 0): 0.036, (1, 1, 1, 0): 0.144, (0, 0, 0, 1): 0.144, (0, 0, 1, 1): 0.036, (0, 1, 0, 1): 0.024, (0, 1, 1, 1): 0.096, (1, 0, 0, 1): 0.224, (1, 0, 1, 1): 0.056, (1, 1, 0, 1): 0.084, (1, 1, 1, 1): 0.336}


P_obs_dict = P_ZAMY

In [3]:
# get P_obs (P_ZAMY) as a numpy array with coordinates [z,a,m,y]
P_obs = np.array([[[[P_obs_dict[(z,a,m,y)] for y in range(cardY)] for m in range(cardM)] for a in range(cardA)] for z in range(cardZ)])

In [4]:
A_from_Z_strategies = np.vstack(tuple(np.ndindex(tuple(np.broadcast_to(cardA, cardZ)))))
cardUAY = len(A_from_Z_strategies)

M_from_A_strategies = np.vstack(tuple(np.ndindex(tuple(np.broadcast_to(cardM, cardA)))))

Y_from_M_and_UAY_strategies = np.vstack(tuple(np.ndindex(tuple(np.broadcast_to(cardY, cardM*cardUAY)))))
Y_from_M_and_UAY_strategies = Y_from_M_and_UAY_strategies.reshape((-1, cardM, cardUAY))

shapeUMY = (len(M_from_A_strategies), len(Y_from_M_and_UAY_strategies))

In [5]:
# env = gp.Env(empty=True)
# env.setParam('LogToConsole', False) # To supress output
# env.start()
m = gp.Model("qcp")
m.params.NonConvex = 2  # Using quadratic equality constraints.
# m.params.LogFile = "solver_output.txt"
# m.setParam('OptimalityTol', 0.01)


W_UAY = m.addMVar(cardUAY, lb=0, name="u_AY")
m.addConstr(W_UAY.sum()==1)
W_UMY = m.addMVar(shapeUMY, lb=0, name="u_MY")
m.addConstr(W_UMY.sum() == 1)
observable_internal = gp.MQuadExpr.zeros((cardZ, cardA, cardM, cardY))
do_internal = gp.MQuadExpr.zeros((cardA, cardY))
m.update()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-09
Set parameter NonConvex to value 2


In [6]:
for A_from_Z_idx, A_from_Z_strategy in enumerate(A_from_Z_strategies):
    w_UAY = W_UAY[A_from_Z_idx].item()
    UAY_val = A_from_Z_idx
    for M_from_A_idx, M_from_A_strategy in enumerate(M_from_A_strategies):
        for Y_from_M_and_UAY_idx, Y_from_M_and_UAY_strategy in enumerate(Y_from_M_and_UAY_strategies):
            w_UMY = W_UMY[M_from_A_idx, Y_from_M_and_UAY_idx].item()
            weight = w_UAY * w_UMY
            for z in range(cardZ):
                a_obs = A_from_Z_strategy[z]
                m_obs = M_from_A_strategy[a_obs]
                y_obs = Y_from_M_and_UAY_strategy[m_obs, UAY_val]
                observable_internal[z, a_obs, m_obs, y_obs] += weight
            for a_sharp in range(cardA):
                m_do = M_from_A_strategy[a_sharp]
                y_do = Y_from_M_and_UAY_strategy[m_do, UAY_val]
                do_internal[a_sharp, y_do] += weight

## Execute:

In [10]:
(y,b) = (1,1)

In [12]:
#%%capture

m.addConstr(P_obs == observable_internal)

objective = do_internal[y,b]
m.setObjective(objective, sense=GRB.MAXIMIZE)
m.optimize()
maximal = do_internal[y,b].getValue()


objective = do_internal[y,b]
m.setObjective(objective, sense=GRB.MINIMIZE)
m.optimize()
minimal = do_internal[y,b].getValue()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 2 rows, 1028 columns and 1028 nonzeros
Model fingerprint: 0x271119d2
Model has 2048 quadratic objective terms
Model has 32 quadratic constraints
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
  QRHS range       [2e-02, 3e-01]



Continuous model is non-convex -- solving as a MIP


MIP start from previous solve produced solution with objective 0.72 (1.26s)
Loaded MIP start from previous solve with objective 0.72
Processed MIP start in 1.25 seconds (1.58 work units)

Presolve time: 0.02s
Presolved: 16403 rows, 5125 columns, 43013 nonzeros
Presolved model has 4096 bilinear constraint(s)
Variable types: 5125 continuous, 0 integer (0 binary)

Root relaxation: cutoff, 5 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0         0.72000    0.72000  0.00%     -    1s

Explored 1 nodes (5 simplex iterations) in 1.58 seconds (1.71 work units)
Thread count was 16 (of 16 available processors)

Solution count 1: 0.72 
No other solutions better than 0.72

Optimal solution found (tolerance 1.00e-04)
Best objective 7.200000000000e-01, best bound 7.2000000

In [13]:
print("min:", minimal, "\nmax:", maximal, "\nrange:", maximal-minimal)

min: 0.41999999999999954 
max: 0.7200000000000416 
range: 0.30000000000004207
